In [ ]:
class Grams:
    def __init__(self):
        from threading import Lock
        self._counter = 0
        self._lock = Lock()

    def __call__(self) -> str:
        with self._lock:
            result = self._num_to_alpha(self._counter)
            self._counter += 1
            return result

    def _num_to_alpha(self, num: int) -> str:
        result = ''
        while num >= 0:
            result = chr(65 + (num % 26)) + result
            num = num // 26 - 1
        return result

# Example usage:
grams = Grams()
for _ in range(30):
    print(grams())

In [ ]:
class Counter():
    def __init__(self):
        from threading import Thread
        from threading import Lock
        self._counter = 0
        self._lock = Lock()
    def as_int(self)->int:
        with self._lock:
            return self._counter
    def next_int(self)->int:
        with self._lock:
            self._counter += 1
            return self._counter
epoch = Counter()
counter = Counter()

In [ ]:
def names(nodes):
    return [node.name for node in nodes]
def show(nodes):
    text = '---\n'
    for node in nodes:
        text+=f'{node.name}\n\tI:{node.I}\t O:{node.O}\n'
    text += '---'

In [ ]:
import cProfile
cProfile.run('cell.reset()()')

In [ ]:
def is_subset_of(little, big):
    if is_iterable(little):
        for l in little:
            if not is_subset_of(l,big):
                return False
        return True
    if is_iterable(big):
        return little in big
    if little == big:
        return True
    try : 
        return isinstance(little,big)
    except:
        return False

def test():
    assert is_subset_of([2,3],[2,3])
    assert not is_subset_of([2,9],[2,3])
    assert is_subset_of(2,[2,3])
    assert is_subset_of(2,range(0,6))
    assert not is_subset_of(2.5,range(0,6))
    assert is_subset_of(range(0,6), int)
    assert not is_subset_of(9,[2,3])
    assert is_subset_of(2,int)
    assert is_subset_of(2,int|bool)
    assert not is_subset_of(2,str|bool)
    # assert is_subset_of({lambda : 23, lambda x: (x,x), int}, set)
test()
subset_of = Infix(is_subset_of)
# not_of = Infix(lambda x,T : not is_subset_of(x,T))

In [21]:
# ITERABLE OPERATIONS

def iterable_from(data):
    try:
        iter(data)
    except :
        return None
    if isinstance(data, str):
        return    
    if isinstance(data, set):
        return set
    if isinstance(data, tuple):
        return tuple
    if isinstance(data, list):
        return list
    if isinstance(data, MorphismType):
        return lambda I_and_O: MorphismType(I_and_O[0], I_and_O[1])
    if isinstance(data, ArrayType):
        return lambda T: ArrayType(data, T)
    raise RuntimeError(f'handle that case pal: {data}, of type : {type(data)} what kinda iterable it is ?')
def test():
    assert(iterable_from(tuple((2,3,4,5,))) == tuple)
    assert(iterable_from(set((2,3,4,5,))) == set)
    assert(iterable_from(list((2,3,4,5,))) == list)
    assert(iterable_from(list((2,3,4,5,list((2,3,4,))))) == list)
    assert(iterable_from([int,bool])==list)
test()
# assert(iterable_from(MorphismType((int,),bool)) == Morphism)

# def apply(data, on_list=lambda a_list : a_list):
#     iteratable = iterable_from(data)
#     if iteratable is not None:
#         return iteratable( on_list(list(element for element in data)) )
#     raise RuntimeError(f'need am iterable as first argument but got: {data}, of type : {type(data)}')

def on_each(data, f): #-> data
    to_iterable = iterable_from(data)
    if to_iterable is not None:
        return to_iterable( on_each(element,f) for element in data )
    if to_iterable is None:
        return f(data)
    raise RuntimeError(f'handle that case buddy : {data}')

assert(on_each(set(('d',4,2342.34)), lambda x : (x, x)) == set((('d','d'),(4,4),(2342.34,2342.34)) ) )

def accrue(data, accu, accu_from_accu_and_element): # f (accu/init, element) -> one element
    to_iterable = iterable_from(data)
    if to_iterable is not None:
        list(accu := accu_from_accu_and_element(accu, elem) for elem in data)
        return accu
    return accu_from_accu_and_element(acc, data)

assert(4321 == (accrue([20, 300, 4000], 1, lambda accumulator, element : accumulator + element)))

def walk(data): # f (accu/init, element) -> one element
    to_iterable = iterable_from(data)
    if to_iterable is not None:
        for element in data:
            yield from walk(element)
    if to_iterable is None:
        yield data

generator = walk([20, 300, 4000,['la', 'asdwerew',True]])
assert(list(generator) == [20, 300, 4000,'la', 'asdwerew',True])

def is_one_inside(data, tester):
    for element in walk(data):
        if tester(element):
            return True
    return False

def eq_composite(data_left,data_right): # f (left, right) -> None
    return data_left == data_right

assert(eq_composite(
    (0,1,2,3,(11,12,13,14,(21,22,23))),
    (0,1,2,3,(11,12,13,14,(21,22,23))),
))
assert(not eq_composite(
    (0,1,2,3,(11,12,13,14,(21,22,23))),
    (0,1,2,3,(11,12,13,14,(21,9999,23))),
))
assert(not eq_composite(
    (0,1,2,3,(11,12,13,(14,21,22,23))),
    (0,1,2,3,(11,12,13,14,(21,24,23))),
))

def structure_of_its_types( primitive_or_composite_value):
    iteratable = iterable_from(primitive_or_composite_value)
    if iteratable is not None:
        return iteratable( (T(value) for value in primitive_or_composite_value) )
    return type(primitive_or_composite_value)

T = lambda data: on_each(data,type)

T(tuple(('d',4,2342.34,['lal',True,False])))

Noning = lambda data : on_each(data, lambda _ : None)

def set_on_iterable(an_iterable, index_or_value, new_value):
    iteratable = iterable_from(an_iterable)
    if iteratable is set:
        old_value = index_or_value
        an_iterable.remove(old_value)
        an_iterable.add(new_value)
        return an_iterable
    index = index_or_value
    if iteratable is tuple:
        as_a_list = list(an_iterable)
        as_a_list[index] = new_value
        return iteratable(as_a_list)
    if iteratable is list:
        an_iterable[index] = new_value
        return an_iterable
    raise RuntimeError(f'is that an iterable: {an_iterable} ????')
def test():
    assert(
        set_on_iterable(tuple((2,3,4,bool)),1,9.1)
        == tuple((2,9.1,4,bool,))
    )
    
    assert(
        set_on_iterable(list((2,3,4,bool)),1,9.1)
        == list((2,9.1,4,bool,))
    )
    assert(
        set_on_iterable(set((2,3,3,bool)),2,9.1)
        == set((9.1,3,3,bool,))
    )
test()

In [ ]:
class DummyType(type):
    def __new__(a_class , name,  bases=(), namespace={}):
        return super().__new__(a_class , name, bases, namespace )
    def __init__(a_class , name=None, bases=(), namespace={}):
        pass
    def __eq__(a_class , other):
        if not isinstance(other,type):
            return False
        if not isinstance(other,DummyType):
            return False
        return a_class .__name__ == other.__name__
    def is_variable(a_class ):
        return True
    def __str__(a_class ):
        return a_class .__name__
    def __repr__(a_class ):
        return a_class .__name__
    def __hash__(a_class ):
        return hash(a_class .__name__)

def test():
    npubyte =  DummyType('np.ubyte')
    assert not npubyte == np.ubyte
    assert npubyte == DummyType('np.ubyte')
    assert not npubyte == DummyType('np.ubye')
    assert isinstance(npubyte,type)
    assert npubyte |eq| AnyType()
    assert AnyType() == npubyte
test()

In [ ]:
import matplotlib.pyplot as plt

def plot_sequences(xs, *yss):
    markers = ['x','+','|','_','1','2','3','4']
    plt.figure(figsize=(10, 6))
    for ys, marker in zip(yss,markers):
        
        plt.scatter(xs, ys, marker=marker,alpha=0.5)
    plt.title('Matplotlib Plot')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.grid(True)
    plt.legend([f'Curve {i+1}' for i in range(len(yss))])
    plt.show()


In [ ]:
pprint = import_with_pip('pprint')
import pprint
pp = pprint.PrettyPrinter(indent=4)
pretty = pp.pprint
pp.pprint([1,2,3,4,5,[0,9,8,7,6,5,[4,7,5,6]]])

In [ ]:
import time
def time_it(a_function, timer):
    def innie(*args, **kargs):
        start = time.perf_counter_ns()
        result = a_function(*args, **kargs)
        timer[0] = time.perf_counter_ns() - start
        return result 
    return innie

def time_that(a_function):
    def innie(*args, **kargs):
        start = time.perf_counter_ns()
        result = a_function(*args, **kargs)
        time.perf_counter_ns() - start
        return result, time.perf_counter_ns() - start
    return innie

In [ ]:
timer = [None]
timeit(cell.reset(), timer)
print(f'{timer[0] = }')
timeit(lambda : 9+4, timer)
print(f'{timer[0] = }')
def truc():
    neuf, quatre = 9, 4
    couple = (9,4)
    add = lambda ab: ab[0] + ab[1]
    apply = lambda fx: fx[0](fx[1])
    add_and_couple = (add,(9,4))
    return apply(add_and_couple)
timeit(truc, timer)
print(f'{timer[0] = }')

In [ ]:
from functools import cached_property
class Node:
    def __init__(self, input_type, output_type, data, name=None):
        self.I = input_type
        self.O = output_type
        self.inputers = tuple() if input_type is None else tuple(None for sub_I in input_type)
        self.inputees = set()
        self.data = data
        self.name = name if name is not None else random_word(6)
        
    @cached_property
    def output(self):
        # print(f'output the {self.name}')
        if not (callable(self.data)):
            return self.data
        if self.I is None: 
            return self.data()
        if self.I is not None:
            # print(f'inputers the {self.inputers}')
            # print(f'inputer the {[node.name for node in self.inputers]}')
            for inputer in self.inputers:
                if inputer is None: raise RuntimeError(f'inputer is none {self.inputers}')
            inputs = tuple(inputer.output for inputer in self.inputers)
            # print(f'output the {self.name}')
            # print(f'{inputs = }')
        output = self.data(inputs)
        return output
        
    def reset(self):
        self.__dict__.pop('output', None)
        return self
        
    def set_data(self, new_data):
        self.data = new_data
        self.reset()
        return self

    def link(self, index, inputer):
        assert(self.inputers[index] == None) # unlink first
        assert(inputer.O != None) # can't link a terminator node
        assert(inputer.O == self.I[index]) # type missmatch
        # reference inputer
        self.inputers = set_on_iterable(self.inputers, index, inputer)
        inputer.inputees.add(self)
        # inference if type parameters
        infer_from_left(
            a_type_value=inputer.O,
            a_parametrized_type_value=self.I[index],
            to_be_infered_type_value=self.I,
        )
        infer_from_left(
            a_type_value=inputer.O,
            a_parametrized_type_value=self.I[index],
            to_be_infered_type_value=self.O,
        )
        assert(self.inputers[index] is not None)
        assert(self in inputer.inputees)
        # print(f'node {self.name} is linked to {inputer.name}')
        return self
        
    def unlink(self, index):
        inputer = self.inputers[index]
        self.inputers = set_on_iterable(self.inputers, index, None)
        # print(f'node {self.name} is unlinked from {inputer.name}')
        if inputer not in self.inputers: # in case there was not multiple links between the nodes
            inputer.inputees.remove(self)
        return self

    def walk_down(self):
        yield self
        for inputer in self.inputers:
            yield from inputer.walk()

    def walk_up(self):
        yield self
        for inputee in self.inputees:
            yield from inputee.walk()

    def lineage(self, end_node=None):
        lineage = set() # of uids
        to_dos = set() # of uids
        looping_node = self
        while True:
            if node != end_node:
                for inputee in looping_node.inputees: #add all parents to todo list
                    lineage.add(inputee)
                    to_dos.add(inputee)
            try:
                looping_node = to_dos.pop() ## next in the list
            except:
                pass 
            if not to_dos and not looping_node.inputees: # we are done
                return lineage

    def ancestors(self, end_node=None):
        ancestors = set() # of uids
        to_dos = set() # of uids
        looping_node = self
        while True:
            if node != end_node:
                for inputer in looping_node.inputers: #add all parents to todo list
                    ancestors.add(inputer)
                    to_dos.add(inputer)
            try:
                looping_node = to_dos.pop() ## next in the list
            except:
                pass 
            if not to_dos and not looping_node.inputers: # we are done
                return ancestors
                
    def matching_inputees(self, some_inputees):
    # we select nodes whose all individual input (sub) types pair with one of the provided types
        kept_nodes = []
        for inputee in some_inputees:
            does_it_match = False
            for sub_I in inputee.I:
                for a_type in some_types:
                    does_it_match = does_it_match or (sub_I == self.O)
            if does_it_match:
                kept_nodes.append(inputee)
        return kept_nodes   

    def matching_inputers(self, index, some_inputers):
    # we select nodes whose all individual input (sub) types pair with one of the provided types
        kept_nodes = []
        sub_I = self.I[index]
        for inputer in some_inputers:
            if inputer.O == sub_I:
                kept_nodes.append(inputee)
        return kept_nodes

In [ ]:
b = 5
a = [1,2,3,4,5]
def loop(thing):
    try:
        yield from (element for element in thing)
    except:
        yield thing

for v in loop(b):
    print(v)
len(loop(a))

In [ ]:
a = [1,2,3,4,5]
b = 5
def iterate(thing):
    if thing |is_of| Iterable:
        return (element for element in thing)
    yield thing
for v in [*b]:
    print(v)


In [ ]:
# more type stuff

def infererences_from_left(data_left,data_right):
    if isinstance(data_right, AnyType) and not isinstance(data_left, AnyType):
        yield (data_left,data_right,)
    if not isinstance(data_right, AnyType):
        to_iterable_left = iterable_from(data_left)
        to_iterable_right = iterable_from(data_right)
        if to_iterable_left != to_iterable_right:
            raise RuntimeError(f'different shapes {data_left = } {data_right = }')
        if to_iterable_left is not None:
            for (element_left, element_right) in zip(data_left, data_right):
                yield from infererences_from_left(element_left, element_right)
    # if to_iterable is None:
    #     yield data


A = Any('A')
assert(
    list(infererences_from_left(
        (int,int,bool,(float,bool,int,)),
        (int,int,bool, A,),
    )) 
    == [((float,bool,int,),A)]
)
B = Any('B')
assert(
    list(infererences_from_left(
        (int,int,bool,(float,bool,int,)),
    (int,int,bool,(float, bool, B)),
    )) 
    == [(int, B)]
)
assert(
    list(infererences_from_left(
        (int,int,bool,(float,bool,int,)),
    (int,int,bool,(float, bool, B)),
    )) 
    != [(bool, B)]
)

assert(
    list(infererences_from_left(
        (int,int,bool,(float,bool,int,)),
    (int,int,A,B),
    )) 
    == [(bool, A),((float,bool,int,), B),]
)
assert(
    list(infererences_from_left(
        (int,int,bool,(float,bool,int,)),
    (int,int,bool,(float, bool, 9)),
    )) 
    == []
)

def infer_from_one_inference(a_type_value, an_inference):
    concrete_type = an_inference[0]
    parameter_type = an_inference[1]
    def replace_if(value, comparator, new_value):
        test = new_value if value is comparator else value
        # print(f'replacing {value} with {new_value} => {test}')
        return new_value if value is comparator else value
    return on_each(a_type_value, lambda a_value: replace_if(a_value, parameter_type, concrete_type))

assert(
    infer_from_one_inference(
        a_type_value=(int,int,A,B),
        an_inference=(bool, A),
    )
    == (int,int,bool,B)
)

def infer_from_inferences(a_type_value, some_inferences):
    for infer in some_inferences:
        a_type_value = infer_from_one_inference(a_type_value, infer)
    return a_type_value

assert(
    infer_from_inferences(
        a_type_value=(int,int,A,B),
        some_inferences=[(bool, A),((int,bool,float,),B)],
    )
    == (int,int,bool,(int,bool,float,))
)

def infer_from_left(a_type_value, a_parametric_type_value, to_be_infered_type_value):
    inferences = infererences_from_left(
        a_type_value, 
        a_parametric_type_value,
    )
    infered = infer_from_inferences(
        a_type_value=to_be_infered_type_value,
        some_inferences=inferences,
    )
    return infered

assert(
    infer_from_left(
        a_type_value=(int,int,A,(float,bool,int,)),
        a_parametric_type_value=(int,int,int,B),
        to_be_infered_type_value=(int,int,int,B,bool),
    )
    == (int,int,int,(float,bool,int,),bool)
)
assert(
    infer_from_left(
        a_type_value=(int,Any('x'),A,(float,bool,int,)),
        a_parametric_type_value=(int,int,Any(''),B),
        to_be_infered_type_value=(int,int,Any(''),B,float),
    )
    == (int,int,A,(float,bool,int,),float)
)
T = Any(grams())
assert(
    infer_from_left(
        a_type_value=int,
        a_parametric_type_value=T,
        to_be_infered_type_value=T,
    )
    == int
)
I = Any('i')
B = Any('b')
full_type = N(
    I=(int,I,B,),
    O= const_1 |q| B,
)
print(
    infer_from_left(
        (int,int,bool,),
        full_type.I,
        full_type.O(full_type.I),
    )
)


In [ ]:
# def infer(node, index, inferer:type):
#     assert(node.inputers[index] == None) # cannot infer linked node slot
#     assert(inferer |not_of| AnyType) # let's keep it simple
#     assert(is_parametric(node.Is[index])) # no useless op
#     assert(inferer |eqeq| node.Is[index]) # type missmatch
#     node.O =infer_from_left(
#         a_type_value=inferer,
#         a_parametric_type_value=node.Is[index],
#         to_be_infered_type_value=node.O,
#     )
#     node.Is = infer_from_left(
#         a_type_value=inferer,
#         a_parametric_type_value=node.Is[index],
#         to_be_infered_type_value=node.Is,
#     )
#     return node

In [ ]:
class TypeVariable(type):
    def __new__(cls, name, set_of_types=None):
        new_type_variable = super().__new__(cls, name, (object,), {})
        new_type_variable.bounds = Any('any') if set_of_types is None else set_of_types
        return new_type_variable
    def infer(cls, inferer):
        assert inferer |of| type, f'can only infer types and not {inferer = }'
        if inferer |of| TypeVariable:
            
        if not inferer |of| TypeVariable:
            
            
        return cls.bounds.__repr__
    def __eq__(self, other):
        if other |of| TypeVariable:
            return other.bounds == self.ounds
        return False
    def __str__(cls):
        return cls.bounds.__str__
    # def __repr__(cls):
    #     return cls.bounds.__repr__
    # def __hash__(cls):
        # return hash(cls.__name__)
        
def T(name=None, set_of_concrete_types=None):
    name = grams() if name is None else name
    new_type_variable = TypeVariable(name, set_of_concrete_types)
    return new_type_variable

type_var = T('a type var')
print(f'{type_var = }')
print(f'{type_var.bounds = }')


In [ ]:
def zip_tensors(tensors, checks):
    def recursive_zip(elements, depth):
        if depth == 0 or not isinstance(elements[0], list):
            return list(zip(*elements))
        return [recursive_zip([elem[i] for elem in elements if i < len(elem)], depth - 1) 
                for i in range(max(len(e) for e in elements))]

    max_depth = max(ground_types)
    result = recursive_zip(tensors, max_depth)
    return result

tensors = ([[1, 2, 3], [10, 20, 30]], [4, 5, 6])
ground_types = [1, 0]
# assert zip_tensors(tensors, ground_types) == [(1, 4), (2, 5), (3, 6)]

zip_tensors(
    (
        [[1,11,111],[2,22,222],[3,33,333]],
        [["1","11","111"],["2","22","222"],["3","33","333"]],
    ),
)
# tensors = [[[[1, 2], [3, 4]], [[5, 6], [7, 8]]], [[[9, 10], [11, 12]], [[13, 14], [15, 16]]]]
# ground_types = [2, 2]
# assert zip_tensors(tensors, ground_types) == [[[(1, 9), (2, 10)], [(3, 11), (4, 12)]], [[(5, 13), (6, 14)], [(7, 15), (8, 16)]]]
# for t in tensors:
#     print(f'{t = }')
# zip_tensors(tensors, ground_types)




In [ ]:
def broadcast_shape(shape_left, shape_right):  
    assert are_shapes_broadcastable(shape_left, shape_right)
    rev_left = reversed(shape_left)
    rev_right = reversed(shape_right)
    result = []
    for l,r in zip(rev_left, rev_right):
        if not l |eq| r:
            1/0
        if is_variable(l):
            result.append(r)
        else:
            result.append(l)
    try:
        len(shape_left), len(shape_right)
        
    while True:
        rev_left[counter]
        rev_right[counter]
        if is_variable(l):
            result.append(r)
        else:
            result.append(l)
        counter +=1
    return result
1/0

    # try :
    #     longest_shape = shape_left if len(shape_left) > len(shape_right) else shape_right
    #     return tuple(reversed


In [ ]:

def all_shapes_and_subtypes_from_tensorType(T):
    print(f'{T = }')
    full_shape, lowest_type = T.shape, T.subtype
    print(f'{full_shape = }')
    print(f'{lowest_type = }')
    all_prefixes, all_suffixes = [], []
    for l in full_shape:
        all_prefixes.append(full_shape[:l])
        all_suffixes.append(full_shape[l-1:])
    # all_supertypes = [ArrayType(prefix,AnyType()) for prefix in all_prefixes]
    all_subtypes = [TensorType(suffix, lowest_type) for suffix in all_suffixes]
    return all_prefixes, all_subtypes
    
# all_possible_supertypes_and_subtypes_from_tensorType(ArrayType((1,2,3,4), bool))
all_shapes_and_subtypes_from_tensorType(TensorType((1,2,3,4,5,6,7,), bool))

    

In [ ]:
# def slide_function_by_patches_over_array(
#     input_array, 
#     patch_shape, 
#     output_element_from_patch, 
#     array_maker=lambda xs: list(xs)
# ):
#     input_shape = shape_from_array(input_array)
#     output_shape = [input_shape[i] - patch_shape[i] + 1 for i in range(len(patch_shape))]
#     output_array = zeroes_from_shape(output_shape, None, array_maker)
#     assert all([i >= p for i,p in zip(input_shape, patch_shape)])
#     indices = [0 for _ in patch_shape]
#     def slide_recursive(
#         output, # output that element will be updated by indices
#         func=lambda x:x,  # function to opearate on patch whose output get into output 
#         indices=[], # the indices that loop through at patch shape level
#         level=0, # depth into the array
#         array_maker = lambda xs: list(xs)
#     ):
#         loop into the output
#         print(f'{output = }')
#         print(f'{indices = }')
#         print(f'{level = }')
#         output_shape = shape_from_array(output)
#         print(f'{output_shape = }')
#         if level > len(output_shape):
#             1/0
#         if level == len(output_shape): # we reached the lowest depth
#             patch = get_patch(input_array, indices, patch_shape, array_maker)
#             # we get the patch at the coordinates
#             print(f'  {patch = }')
#             sub_array = output
#             for index in indices[:-1]: # we loop into the indices in reverse
#                 sub_array = sub_array[index] # we get down one level
#             sub_array[indices[-1]] = func(patch)
#         else:
#             for i in range(output_shape[level]):
#                 slide_recursive(
#                     output, 
#                     func, 
#                     [*indices,i], 
#                     level+1, 
#                     array_maker,
#                 )

#     slide_recursive(
#         output_array, 
#         output_element_from_patch,
#         level=0,
#         array_maker=array_maker,
#     )
#     return output_array

# # result = slide_function_by_patches_over_array(
# #     input_array=[
# #         [[-991,2], [2,2], [3,2], [4,9]],
# #         [[5,2], [6,2], [7,2], [8,45]],
# #         [[9,2], [10,2], [11,2], [12,7]],
# #         [[13,2], [14,2], [15,2], [16,900]],
# #     ], 
# #     patch_shape=(2, 2), 
# #     output_element_from_patch=lambda patch: np.multiply(patch, np.array([[0,1],[-1,0]])).sum(), 
# #     array_maker=lambda xs: np.array(xs),
# # )
# # print(result)

# result = slide_function_by_patches_over_array(
#     input_array=[
#         [[-991,2], [2,2], [3,2], [4,9]],
#         [[5,2], [6,2], [7,2], [8,45]],
#         [[9,2], [10,2], [11,2], [12,7]],
#         [[13,2], [14,2], [15,2], [16,900]],
#     ], 
#     patch_shape=(2, 2), 
#     output_element_from_patch=lambda patch: patch, 
#     # array_maker=lambda xs: np.array(xs),
# )
# print(result)



In [ ]:
def mulitply_patch_node(array_maker=lambda xs: list(xs), name=None):
    arr = array_maker
    take a tensor patch [s]T
    patch_type = ArrayType( (AnyInt(), AnyInt(), ), AnyValue() )
    middle_type = {patch_shape}
    def Is_from_node(node): # [h][w]T
        return (node.inferences{patch_type})
    def O_from_Is(Is): # \ [h][w]T -> [h][w]T /
        return MorphismType(
            Is[0],
            lamda X: X,
        )
    def infering(node, index, inferer):
        assert index == 0
        assert isinstance(inferer, ArrayType)
        assert inferer.subtype.subtype
        if not node.inferences[patch_type] == inferer:
            1/0
        if node.inferences[patch_type] == inferer:
            
        coordinates = [(index := index // dim, index % dim)[1] for dim in reversed(shape)][::-1]
        shape |apiece| (lambda x: x |check| int)
        while True: # check coordinates validity
            if coordinates |instance_of| int and shape |instance_of| int:
                assert coord < dim, f'{coord = } does not fit {dim = }'
                break
            for c,d in zip(coordinates,shape):
                assert c < d, f'{c = } does not fit {d = }'
            break
        assert inferer |eq| node.inferences[T]
        assert inferer |subset_of| node.inferences[T]
        node.inferences[T] = inferer
    def data( xss ):
        def innie( yss ):
            zss = arr([ 
                arr([ x*y for x,y in zip(xs,ys) ])
                for xs,ys in zip(xss,yss) 
            ])
            for xs, ys in zip(xss,yss):
                zs = arr([x*y for x,y in zip(xs,ys)])
                    
            return xs * ys
        return innie
    return Node(
        middle_types={T},
        infering_from_index_and_type=infering,
        Is_from_node=Is_from_node,
        O_from_Is=O_from_Is,
        output_from_node_and_inputs=lambda xs: reshape(xs, shape, array_from_list),
        name=f'reshaper {ArrayType(shape,T)}' if name is None else name,
    )

# bsss_array = reshaper_node(shape=(2,3,4), T=AnyType())
# print(f'{bsss_array.Is() = }')
# # print(f'{bsss_array.inputers = }')
# # print(f'{bsss_array.inferences = }')
# for i in range(2*3*4):
#     an_int_node = param_node(int, random.randrange(20), f'p{i}')
#     bsss_array.infer(i,an_int_node.O(_))
#     bsss_array.link(i,an_int_node)

# # print(f'{bsss_array.O(bsss_array.Is()) = }')
# print(f'{bsss_array() = }')

# # print(f'{type(output) = }')
# # print(f'{type(output[0]) = }')
# # print(f'{type(output[0][0]) = }')
# # print(f'{type(output[0][0][0]) = }')
# # print(f'{type(output[0][0][0][0]) = }')

In [ ]:
# def point_extension_node(array_maker=lambda xs: list(xs),name=None):
#     # extend function to higher dimension
#     # | \T->Y/ -> \[n]T -> [n]Y/ /
#     n = AnyInt()
#     T, Y, = AnyType(), AnyType(),
#     TtoY = MorphismType(T, lambda _:Y)
#     middle_types = {TtoY}
#     # assert T |instance_of| type, f'{T = } must be of type'
#     # dimensions |apiece| (lambda x: x |check| int)
#     def infering_T_to_Y(node, index, T_to_Y):
#         assert T_to_Y |instance_of| MorphismType, f'{T_to_Y = } not a Morphism'
#         assert index == 0, f'{index = } must be a 0 integer'
#         current_I = node.inferences[TtoY].I
#         current_O = node.inferences[TtoY].O(current_I)
#         # print(f'{T_to_Y = }')
#         # print(f'{current_I = }')
#         # print(f'{current_O = }')
#         # print(f'{current_I|eq|T_to_Y.I = }')
#         # print(f'{current_O|eq|T_to_Y.O = }')
#         # print(f'{current_O == T_to_Y.O(T_to_Y.I) = }')
#         assert T_to_Y.I |eq| current_I, f'{T_to_Y.I = } not compatible {current_I = }'
#         assert T_to_Y.O(T_to_Y.I) |eq| current_O, f'{T_to_Y.O(T_to_Y.I) = } not compatible {current_O = }'
#         node.inferences[TtoY] = T_to_Y
 
#     def Is_from_node(node):
#         return (node.inferences[TtoY],)
        
#     def O_from_Is(Is):
#         # Is expected to be  \[]T -> \_>[]Y/ /
#         return MorphismType(
#             ArrayType(n, Is[0].I),
#             lambda _arr_I : ArrayType(n,Is[0].O(Is[0].I)),
#         )
#     def extension_of_f(function_in_single):
#         function = function_in_single[0]
#         def list_of_result_from_inputs(inputs):
#             result_list = array_maker([function(x) for x in inputs])
#             return result_list
#         return list_of_result_from_inputs

#     return Node(
#         middle_types=middle_types,
#         infering_from_index_and_type=infering_T_to_Y,
#         Is_from_node=Is_from_node,
#         O_from_Is=O_from_Is, 
#         output_from_node_and_inputs=extension_of_f,
#         name=f'{TtoY} to {MorphismType(ArrayType(n,T), lambda _: ArrayType(n,Y))}' if name is None else name,
#     )

# extender_node = point_extension_node()
# f_node = const_node(F(int, lambda _: float), lambda x: x**(2/3))
# print(f'before inference: {extender_node.Is() = }')
# print(f'{extender_node.O(extender_node.Is()) = }')
# extender_node.infer(0,f_node.O(f_node.Is()))
# extender_node.link(0,f_node)
# print(f'after inference: {extender_node.Is() = }')
# print(f'{extender_node.O(extender_node.Is()) = }')
# # second_extender_node = point_extension_node()
# # second_extender_node.infer(0,extender_node.O(extender_node.Is()))
# # second_extender_node.link(0,extender_node)
# # print(f'after second inference: {second_extender_node.Is() = }')
# # print(f'{second_extender_node.O(second_extender_node.Is()) = }')

# two_three_node = param_node(ArrayType(2,int), np.array([2,3]))
# an_apply_node = applier_node()
# an_apply_node.infer(0,extender_node.O(extender_node.Is()))
# an_apply_node.infer(1,two_three_node.O(two_three_node.Is()))
# an_apply_node.link(0,extender_node)
# an_apply_node.link(1,two_three_node)
# an_apply_node()

In [ ]:
def zip_tensors(tensors, ground_types):
    def is_broadcastable(shape1, shape2):
        return all(s1 == s2 or s1 == 1 or s2 == 1
                   for s1, s2 in zip(reversed(shape1), reversed(shape2)))

    def get_shape(tensor):
        shape = []
        current = tensor
        while isinstance(current, list):
            shape.append(len(current))
            if shape[-1] > 0:
                current = current[0]
            else:
                break
        return tuple(shape)

    def broadcast(tensor, target_shape):
        current_shape = get_shape(tensor)
        if current_shape == target_shape:
            return tensor
        
        # Pad the current shape with 1s to match the target shape length
        current_shape = (1,) * (len(target_shape) - len(current_shape)) + current_shape
        
        def broadcast_recursive(t, current, target):
            if not target:
                return t
            if not isinstance(t, list):
                return [broadcast_recursive(t, current[1:], target[1:])] * target[0]
            if current[0] == 1:
                return [broadcast_recursive(t[0], current[1:], target[1:])] * target[0]
            return [broadcast_recursive(item, current[1:], target[1:]) for item in t]
        
        return broadcast_recursive(tensor, current_shape, target_shape)

    def recursive_zip(elements, depth):
        if depth == 0 or not all(isinstance(e, list) for e in elements):
            return list(zip(*elements))
        
        shapes = [get_shape(e) for e in elements]
        max_shape = tuple(max(s[i] if i < len(s) else 1 for s in shapes) for i in range(max(len(s) for s in shapes)))
        
        if not all(is_broadcastable(s, max_shape) for s in shapes):
            raise ValueError("Tensor shapes are not broadcastable")
        
        broadcasted = [broadcast(e, max_shape) for e in elements]
        return [recursive_zip([elem[i] for elem in broadcasted], depth - 1) 
                for i in range(max_shape[0])]

    # max_depth = max(ground_types)
    result = recursive_zip(tensors, max_depth)
    return result
zip_tensors(
    (
        [[1,11,111],[2,22,222],[3,33,333]],
        [["1","11","111"],["2","22","222"],["3","33","333"]],
    ),
    AnyOfThoseTypes({int,str})
)

In [ ]:
# # take a couple of tensor and broadcast if possible
# def broadcaster_zipper_node():
#     prime_shape = AnyShape()
#     lesser_shape = AnyShape()
#     prime_subtype = AnyType()
#     lesser_subtype = AnyType()
#     def Is_from_node(node): 
#         prime_tensor_type = AnyTensorTypeOf if 
#         the_prime_shape = node.inferences[prime_shape]
#         the_prime_subtype = node.inferences[prime_subtype]
#         if isinstance(the_prime_shape, AnyShape):
#             the_prime_tensor_type = AnyTensorTypeOf(the_prime_subtype)
#         else:
#             the_prime_tensor_type = TensorType(the_prime_shape, the_prime_subtype)
            
#         the_lesser_shape = node.inferences[lesser_shape]
#         the_lesser_subtype = node.inferences[lesser_subtype]
#         if isinstance(the_lesser_shape, AnyShape):
#             the_lesser_tensor_type = AnyTensorTypeOf(the_lesser_subtype)
#         else:
#             the_lesser_tensor_type = TensorType(the_lesser_shape, the_lesser_subtype)
            
#         return ( the_prime_tensor_type, the_lesser_tensor_type )
#     def O_from_Is(Is):
#         la
        
#     A, B, C, = AnyType(), AnyType(), AnyType(),
#     BtoC = MorphismType(B,lambda _: C)
#     AtoB = MorphismType(A,lambda _: B)
    
#     middle_types = {AtoB, BtoC}
#     data = lambda f_g: lambda x:f_g[0](f_g[1](x))
#     # assume that O() is O(Is()) and transfer it to child nodes

#     def infering(node, index, inferer):
#         node.inferences[AtoB]
#         la
  
#     return Node(
#         middle_types=middle_types,
#         infering_from_index_and_type=infering_F_or_G,
#         Is_from_node=Is_from_node,
#         O_from_Is=O_from_Is,
#         output_from_inputs=data,
#         name=f'{"apply" if name is None else name}',

#         middle_types={T},
#         infering_from_index_and_type=infering,
#         Is_from_node=Is_from_node,
#         O_from_Is=O_from_Is,
#         output_from_node_and_inputs=lambda xs: reshape(xs, shape, array_maker),
#         name=f'reshaper {TensorType(shape,T)}' if name is None else name,
#     )


In [ ]:
def generate_indices_tensor(shape):
    def fill_tensor(current_indices, shape):
        if len(current_indices) == len(shape):
            # Base case: return the current indices as the value
            return current_indices
        else:
            # Recursive case: create a list of values for the next dimension
            return [fill_tensor(current_indices + [i], shape) for i in range(shape[len(current_indices)])]

    # Start the recursive filling process
    return fill_tensor([], shape)

# Example usage:
shape = (2, 3, 4)  # A 3D tensor shape
tensor = generate_index_tensor(shape)

# Display the generated tensor
# for layer in tensor:
#     print(layer)
assert (generate_index_tensor((2,3,4)) == 
    [
      [
        [[0, 0, 0], [0, 0, 1], [0, 0, 2], [0, 0, 3]],
        [[0, 1, 0], [0, 1, 1], [0, 1, 2], [0, 1, 3]],
        [[0, 2, 0], [0, 2, 1], [0, 2, 2], [0, 2, 3]]
      ],
      [
        [[1, 0, 0], [1, 0, 1], [1, 0, 2], [1, 0, 3]],
        [[1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 1, 3]],
        [[1, 2, 0], [1, 2, 1], [1, 2, 2], [1, 2, 3]]
      ]
]
)
generate_indices_tensor((2,3)) 


In [ ]:
def iterate_into_indices_from_shapes(shapes):
    max_dimension = len(max(shapes,key=len))
    prime_shape = prime_shape_from_shapes(shapes)
    indicess = zeros_from_shape(prime_shape)
    print(f'{indicess = }')
    for rank, prime_length in enumerate(prime_shape):
        for shape in shapes:
            if len(shape[rank])
        all_lengths_at_one_rank = [ shape[-rank-1] for shape in shapes if rank<len(shape) ]
        print(f'{all_lengths_at_one_rank = }')

iterate_into_indices_from_shapes([[5], [4,5]])
# iterate_into_indices_from_shapes([[3,4,5], [2,3,4,5]])

def iterate_into_indices_from_shapes(shapes):
    if all(len(shape)==1 for shape in shapes):
        lengths = [ shape[0] for shape in shapes ]
        return indices_from_lengths(lengths)
    prime_shape = prime_shape_from_shapes(shapes)
    
    for shape in shapes:
        
    all_lengths_at_one_rank = [ shape[-rank-1] for shape in shapes if rank<len(shape) ]

    if all(len(shape)==1 for shape in shapes):

In [ ]:
def indices_from_lengths(lengths):
    print(f'{lengths = }')
    prime_length = max(lengths)
    for prime_index in range(prime_length):
        print(f'broadcasted indices {[prime_index%l for l in lengths]}')
        yield [ prime_index%l for l in lengths ]


list( indices_from_lengths([8,2,4]) )

In [ ]:
class Shape(tuple):
    def __new__(cls, *lengths):
        for length in lengths:
            if not all(l < 0 or isinstance(l,int) for l in lengths):
                raise Exception(f"{lengths = } should be all naturals")
        return super().__new__(cls, lengths)

    def __repr__(self):
        return f"Shape{tuple(self)}"

    def rank(self):
        return len(self)

    def num_elements(self):
        result = 1
        for dim in self:
            result *= dim
        return result

    def __getitem__(self, key):
        result = super().__getitem__(key)
        if isinstance(key, slice):
            return Shape(*result)
        return result

# Verify that Shape is indeed a type
assert isinstance(Shape, type) == True

# Example usage
shape1 = Shape(2, 3, 4)
print(shape1)  # Output: Shape(2, 3, 4)
print(shape1.rank())  # Output: 3
print(shape1.num_elements())  # Output: 24

# Slicing returns a new Shape
print(shape1[1:])  # Output: Shape(3, 4)

# Invalid shape
try:
    Shape(2, -1, 4)
except ValueError as e:
    print(e)  # Output: All dimensions must be non-negative integers

# Tuple-like behavior
a, b, c = shape1
print(a, b, c)  # Output: 2 3 4

# Iteration
for dim in shape1:
    print(dim)  # Outputs: 2, 3, 4 on separate lines

In [ ]:
import numpy as np 
class CountingNumberMeta(type):
    def __new__(cls, base_type):
        # if not issubclass(base_type, int):
        #     raise TypeError("Base type must be a subclass of int.")
        
        class_name = f"Counting{base_type}"

        # Define a custom class that inherits from base_type
        class CountingNumber(base_type):
            def __new__(cls, value, *args, **kwargs):
                if not isinstance(value, (int, base_type)):
                    raise TypeError(f"Value must be an integer or {base_type.__name__}.")
                if value <= 0:
                    raise ValueError(f"Counting numbers must be greater than zero. Got: {value}.")
                return base_type.__new__(cls, value, *args, **kwargs)

            def _check_result(self,result):
                if result <= 0:
                    raise ValueError(f"Operation result must be greater than zero. Got: {result}.")
                return type(self)(result)
            
            def __add__(self, other):
                if isinstance(other, (int, base_type)):
                    return self._check_result(base_type.__add__(self, other))
                return NotImplemented

            def __sub__(self, other):
                if isinstance(other, (int, base_type)):
                    return self._check_result(base_type.__sub__(self, other))
                return NotImplemented

            def __mul__(self, other):
                if isinstance(other, (int, base_type)):
                    return self._check_result(base_type.__mul__(self, other))
                return NotImplemented

            def __floordiv__(self, other):
                if isinstance(other, (int, base_type)):
                    return self._check_result(base_type.__floordiv__(self, other))
                return NotImplemented

            def __truediv__(self, other):
                if isinstance(other, (int, base_type)):
                    result = base_type.__truediv__(self, other)
                    if result <= 0 or not result.is_integer():
                        raise ValueError(f"Operation result must be a positive integer. Got: {result}.")
                    return cls(int(result))
                return NotImplemented

            def __mod__(self, other):
                if isinstance(other, (int, base_type)):
                    return self._check_result(base_type.__mod__(self, other))
                return NotImplemented

            # Add more operations as necessary, such as __pow__, __lshift__, __rshift__, etc.

        return CountingNumber

# Create specific counting number types using CountingNumberMeta
CountingUbyte = CountingNumberMeta(np.ubyte)
CountingInt = CountingNumberMeta(int)

# Usage Examples
three = CountingUbyte(3)
five = CountingInt(5)
type(three + three)
# print(three + five)  # CountingInt(8)
# print(three - five)  # Raises ValueError: Operation result must be greater than zero.

In [ ]:
def shape(cls, ground_type=None):
    ground_type = cls.subtype if ground_type is None else ground_type
    return shape_from_ArrayType(cls, ground_type=ground_type)
def shape(cls, ground_type=type(None), stop_depth=None):
    def shape_from_ArrayType(array_type, ground_type=type(None), stop_depth=None, shape=[]):
        if array_type |eqeq| ground_type:
            return shape
        if len(shape) == stop_depth:
            return shape
        if not array_type |instance_of| ArrayType:
            return shape
            raise Exception(f'never encountered {ground_type = } but we reached lowest type level {array_type = } {shape = }')
        return shape_from_ArrayType(array_type.subtype, ground_type, stop_depth, shape=shape+[array_type.length])
    return shape_from_ArrayType(cls, ground_type, stop_depth)
